# Baseline method with normalization using Gaussian processes

In this notebook the baseline methiod (normalization -> SVD -> ALS -> denormalization) is extended by adding a initialization of unobserved ratings using Gaussian processes.

The sections "Preprocess", "Run baseline + Gaussion processes method" and "Create submission file" can be run entirely and in sequence to produce the results. Run the "Download data from Kaggle" code block only if you are running on Colab.


## Download data from Kaggle

In [ ]:
!pip install kaggle

!mkdir ~/.kaggle

import json

kaggle_username = "yuvalnis" #@param {type:"string"}
kaggle_api_key = "1800d5a286834f0416c338c7bd7f6dee" #@param {type:"string"}

assert len(kaggle_username) > 0 and len(kaggle_api_key) > 0

api_token = {"username": kaggle_username,"key": kaggle_api_key}

with open('kaggle.json', 'w') as file:
    json.dump(api_token, file)

!mv kaggle.json ~/.kaggle/kaggle.json

!chmod 600 ~/.kaggle/kaggle.json
!kaggle competitions download -c cil-collaborative-filtering-2022

!unzip -n cil-collaborative-filtering-2022.zip

## Preprocess

### Imports

In [ ]:
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split
from typing import Tuple, List

### Declare global parameters of data

In [ ]:
total_num_users = 10000
total_num_movies = 1000

### Data parsing helper function declarations

In [ ]:
def parse_csv(csv_path: str) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
  """
  Extract the arrays of user indices, item indices and ratings listed in a .csv file

  :param csv_path: path to .csv file to read from
  :return: 3 arrays containing the users indices, the item indices and the observed ratings in order  
  """
  df = pd.read_csv(csv_path)
  # extract user and item indices from the Id label in the dataframe
  df = df.join(df.Id.str.extract(r"r(?P<User>\d+)_c(?P<Item>\d+)").astype(int) - 1)
  # extract user, item and prediction triplets from dataframe
  users = df.User.values
  items = df.Item.values
  preds = df.Prediction.values
  return users, items, preds

def construct_ratings_matrix(
    users: np.ndarray,
    items: np.ndarray,
    ratings: np.ndarray,
    n_users: int,
    n_items: int
) -> Tuple[np.ndarray, np.ndarray]:
  """
  Constructs the ratings matrix with NaN values where no rating was observed

  :param users: array of user indices
  :param items: array of item indices
  :param predictions: array of ratings per user-item pair
  :param n_users: total number of users
  :param n_items: total number of items
  :return: the ratings matrix and the observed ratings mask
  """
  ratings_matrix = np.zeros((n_users, n_items))
  observed_mask = np.full((n_users, n_items), fill_value=False)
  for r, c, v in zip(users, items, ratings):
    observed_mask[r, c] = True
    ratings_matrix[r][c] = v

  ratings_matrix[~observed_mask] = np.nan

  return ratings_matrix, observed_mask

### Model function declarations

In [ ]:
def KAA_splicer(covmat, selection_A):
  assert(selection_A.shape[0] == covmat.shape[0])
  assert(selection_A.shape[0] == covmat.shape[1])
  temp = covmat[:, selection_A]
  temp = temp[selection_A, :]
  return temp

def KXA_splicer(covmat, selection_A):
  assert(selection_A.shape[0] == covmat.shape[0])
  assert(selection_A.shape[0] == covmat.shape[1])
  temp = covmat[:, selection_A]
  temp = temp[np.logical_not(selection_A), :]
  return temp

def MUA_splicer(mu, selection_A):
  assert(selection_A.shape[0] == mu.shape[0])
  temp = mu[np.logical_not(selection_A)]
  return temp

def gauss_kernel(a, b, bandwidth):
  return np.exp(-(np.linalg.norm(a-b)**2)/bandwidth**2)

#X is the recommender system Matrix and entry_mask masks the existend entries
def gp_imputation(X, bandwidth, noise):
  #Xcombined = np.concatenate((X, Xtest))
  Xcombined = X
  maskedXcombined = np.ma.array(Xcombined, mask = np.logical_not(np.isnan(Xcombined)))
  mean = np.nanmean(Xcombined, axis = 0)
  cov = np.zeros((Xcombined.shape[1], Xcombined.shape[1]))
  notnan_mask = np.logical_not(np.isnan(Xcombined))
  for i in range(Xcombined.shape[1]):
    localmask = notnan_mask[:, i]
    #cov[i, i] = (np.matmul(Xcombined[localmask, i], Xcombined[localmask, i])**2+1)**2
    cov[i, i] = np.var(Xcombined[localmask, i])
    #cov[i, i] = 1.0
    for j in range(i + 1, Xcombined.shape[1]):
      localmask = notnan_mask[:, i] & notnan_mask[:, j]
      t1 = Xcombined[localmask, i]
      t2 = Xcombined[localmask, j]
      #temp = np.cov([t1, t2])
      #cov[i, j] = (np.matmul(t1, t2) + 1)**2
      #cov[i, j] = temp[0, 1]
      cov[i, j] = gauss_kernel(t1, t2, bandwidth)
      cov[j, i] = cov[i, j]
  Xcombined_new = np.copy(Xcombined)

  print(Xcombined.shape)

  for index, features in enumerate(Xcombined):
    selection_A = notnan_mask[index, :]
    mu = MUA_splicer(mean, selection_A)
    kxa = KXA_splicer(cov, selection_A)
    kaa = KAA_splicer(cov, selection_A)
    ya = Xcombined[index, selection_A]
    #print(mu.shape)
    #print(kxa.shape)
    #print(kaa.shape)
    #print(ya.T.shape)
    pred = mu + np.matmul(kxa, np.linalg.solve(kaa + np.identity(kaa.shape[0]) * noise**2, ya.T))
    #print(pred)
    #assert(False)
    for i in range(Xcombined.shape[1]):
      curr = 0
      if np.logical_not(notnan_mask[index, i]):
        Xcombined_new[index, i] = pred[curr]
        curr += 1
  #assert(not np.any(np.isnan(Xcombined_new)))
  return Xcombined_new


def normalize(data: np.ndarray) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
  """
  Normalizes the input data matrix per item (column). NaN entries are disregarded and are set using Gaussian processes.  

  :param data: matrix to normalize
  :return: the normalized input matrix, the column-wise mean and the column-wise standard deviation
  """
  # compute mean and std and normalized the matrix with NaN values
  mean = np.nanmean(data, axis=0, keepdims=True)
  std = np.nanstd(data, axis=0, keepdims=True)
  normed_data = (data - mean) / std
  # set the non-observed entries to 0
  normed_data = gp_imputation(normed_data, 0.1, 1.0)
  return normed_data, mean, std


def denormalize(data: np.ndarray, mean: np.ndarray, std: np.ndarray) -> np.ndarray:
  """
  Denormalizes the input data matrix per item (column) by performing the inverse operation of the normalize function  

  :param data: matrix to denormalize
  :param mean: the column-wise mean of the unnormalized matrix
  :param std: the column-wise std of the unnormalized matrix
  :return: the denormalized matrix
  """
  return (data * std) + mean


def SVD(A: np.ndarray, k: int) -> np.ndarray:
  """
  Computes the singular value decomposition of the input matrix, projected on
  the subspace defined by the k largest singular values, as a 3-tuple

  :param A: matrix to decompose
  :param k: number of the largest singular values
  :return: the projected singular value decomposition of A
  """
  assert(k <= min(A.shape)), "k should be no greater than the number of users or items."
  U, S, VT = np.linalg.svd(A, full_matrices=False)
  return U[:, :k], np.diag(S[:k]), VT[:k, :]


def RMSE(x: np.ndarray, y: np.ndarray, mask: np.ndarray) -> float:
  return np.sqrt(np.nansum(mask * (x - y) ** 2) / np.sum(mask))


def ALS(
    data: np.ndarray,
    mask: np.ndarray,
    U: np.ndarray,
    VT: np.ndarray,
    n_latent_factors: int,
    regularization_param: float,
    n_iterations: int
) -> np.ndarray:
  regularizer = regularization_param * np.eye(n_latent_factors)

  with tqdm(total=n_iterations * (np.sum(mask.shape))) as pbar:
    rmse_score = RMSE(data, U @ VT, mask)
    pbar.set_description(f"Initial RMSE score is {rmse_score:.4f}")
    for iter in range(n_iterations):
      for i, Ri in enumerate(mask):
        U[i] = np.linalg.solve(
            np.dot(VT, np.dot(np.diag(Ri), VT.T)) + regularizer,
            (np.dot(VT, np.dot(np.diag(Ri), data[i].T))).T
        )
        pbar.update(1)

      for j, Rj in enumerate(mask.T):
        VT[:,j] = np.linalg.solve(
            np.dot(U.T, np.dot(np.diag(Rj), U)) + regularizer,
            np.dot(U.T, np.dot(np.diag(Rj), data[:, j]))
        )
        pbar.update(1)

      rmse_score = RMSE(data, U @ VT, mask)
      pbar.set_description(f"At iteration #{iter + 1} the RMSE score is {rmse_score:.4f}")

  return U @ VT

## Run baseline + Gaussion processes method

In [ ]:
n_latent_factors = 3
regularization_param = 0.1
n_iterations = 20

In [ ]:
users, items, preds = parse_csv('data_train.csv')
ratings, mask = construct_ratings_matrix(users, items, preds, total_num_users, total_num_movies)
norm_data, data_mean, data_std = normalize(ratings)
U, _, VT = SVD(norm_data, n_latent_factors)
normed_pred_ratings = ALS(norm_data, mask, U, VT, n_latent_factors, regularization_param, n_iterations)
pred_ratings = denormalize(normed_pred_ratings, data_mean, data_std)
print(f"Validation RMSE: {RMSE(pred_ratings, ratings, mask)}")

## Create submission file

In [ ]:
submission_users, submission_items, _ = parse_csv('sampleSubmission.csv')
df = pd.DataFrame({
    "Id": [f"r{r + 1}_c{c + 1}" for r, c in zip(submission_users, submission_items)],
    "Prediction": [pred_ratings[r, c] for r, c in zip(submission_users, submission_items)]
})
df.to_csv(f"baseline_gp_reg-{regularization_param:.2f}_k-{n_latent_factors}_iters-{n_iterations}_submission.csv", index=False)